# Psych 198: Reproducibility DeCal (Spring 2021)

## Demo/Lab 1: Hypothesis Testing & Statistical Significance

In this demo/lab, we will go through the basic steps we usually do for hypothesis testing in R, and take a look at some issues with how p-values are reported (excerpt from my personal project).

Author: Yuyang Zhong (2020). This work is licensed under a [Creative Commons BY-NC-SA 4.0 International
License][cc-by].

![CC BY-NC-SA 4.0][cc-by-shield]

[cc-by]: http://creativecommons.org/licenses/by/4.0/
[cc-by-shield]: https://img.shields.io/badge/license-CC--BY--NC--SA%204.0-blue

#### Note on using Jupyter Notebooks 
Enter code into a code cell, then press SHIFT+Enter to run that cell. The output of the code should be shown right underneath the cell you just run.

### Part 1: Running Hypothesis Tests

In this part, we will use the Canadian Women's Labour-Force Participation Dataset to take a look. 

Run the cell below to import the library we need.

In [ ]:
install.packages('tidyverse')

library(car)
library(tidyverse)

Our dataframe is called `Bfox`. Run the cell below to display the first few rows.

In [ ]:
head(Bfox)

#### Simple Linear Regression
`womwage` vs. `tfr` - Women average weekly wages vs. total fertility rate.

In [ ]:
mod1 <- lm(womwage ~ tfr, data=Bfox)
summary(mod1)

**Question:** What is the p-values reported in this relationship? What is the test statistic?

#### Multivariate Linear Regression

`womwage` vs. `tfr` + `parttime` - Women average weekly wages vs. total fertility rate + part time workforce.

In [ ]:
mod2 <- lm(womwage ~ tfr + parttime, data=Bfox)
summary(mod2)

**Question:** What happened here? What became nonsignificant?

#### Interaction Model

`womwage` vs. `tfr` X `parttime` - Women average weekly wages vs. total fertility rate X part time workforce.

In [ ]:
mod3 <- lm(womwage ~ tfr * parttime, data=Bfox)
summary(mod3)

**Question:** What does the interaction model tell us more compared to the multivariate model in the previous section?

### Part 2: Verifying p-values

The following section will be comparing the p-values that was reported in research articles, and a recalculation of the p-value we will compute. Run the cell below to load the data. For simplicity we will only look at F-statistics.

In [ ]:
data <- read.csv("data.csv")
data <- data[data$Type == 'f', ]

head(data)

The following is a way to compute p-value from test statistics and its parameters. Let's look at row 2 of the dataframe.

In [ ]:
data[2,]

We can take out the statistics and the parameters from this row.

In [ ]:
row <- data[2,]

stat <- row$Stat
df1 <- row$Param1
df2 <- row$Param2

pf(stat,df1,df2, lower.tail=F)

**Question:** Is the calculation above correct?

#### Let's apply our calculation to all of the rows.

In [ ]:
recal_data <- data %>%
    rowwise %>%
    mutate(recalculated.p.value = pf(q=Stat, df1=Param1, df2=Param2, lower.tail=F))

head(recal_data)

#### Can we find any rows that have mismatches in p-values?

In [ ]:
recal_data[(recal_data$Reported.p.value - recal_data$recalculated.p.value <= -0.01) & recal_data$Sign != ">", ]